# Overview
The notebook shows how to extract the segmentation map for the ships, augment the images and train a simple DNN model to detect them. A few additional tweaks like balancing the ship-count out a little better have been done.

## Model Parameters
We might want to adjust these later (or do some hyperparameter optimizations)

In [1]:
BATCH_SIZE = 4
EDGE_CROP = 16
NB_EPOCHS = 5
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING = (1, 1)
# number of validation images to use
VALID_IMG_COUNT = 400
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 200
AUGMENT_BRIGHTNESS = False

In [ ]:
!ls ../input

In [28]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.util import montage
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
ship_dir = '../input'
train_image_dir = os.path.join(ship_dir, 'train_v2')
test_image_dir = os.path.join(ship_dir, 'test_v2')
import gc; gc.enable() # memory is tight
os.environ['CUDA_VISIBLE_DEVICES'] = ''
from sklearn.model_selection import train_test_split
import PIL

from skimage.morphology import label
def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)

In [6]:
# masks = pd.read_csv(os.path.join('../input/',
#                                  'train_ship_segmentations_v2.csv'))
# print(masks.shape[0], 'masks found')
# print(masks['ImageId'].value_counts().shape[0])
# masks.head()

# Make sure encode/decode works
Given the process
$$  RLE_0 \stackrel{Decode}{\longrightarrow} \textrm{Image}_0 \stackrel{Encode}{\longrightarrow} RLE_1 \stackrel{Decode}{\longrightarrow} \textrm{Image}_1 $$
We want to check if/that
$ \textrm{Image}_0 \stackrel{?}{=} \textrm{Image}_1 $
We could check the RLEs as well but that is more tedious. Also depending on how the objects have been labeled we might have different counts.



In [7]:
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
# rle_0 = masks.query('ImageId=="00021ddc3.jpg"')['EncodedPixels']
# img_0 = masks_as_image(rle_0)
# ax1.imshow(img_0[:, :, 0])
# ax1.set_title('Image$_0$')
# rle_1 = multi_rle_encode(img_0)
# img_1 = masks_as_image(rle_1)
# ax2.imshow(img_1[:, :, 0])
# ax2.set_title('Image$_1$')
# print('Check Decoding->Encoding',
#       'RLE_0:', len(rle_0), '->',
#       'RLE_1:', len(rle_1))

# Split into training and validation groups
We stratify by the number of boats appearing so we have nice balances in each set

In [9]:
# masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
# unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()
# unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x>0 else 0.0)
# unique_img_ids['has_ship_vec'] = unique_img_ids['has_ship'].map(lambda x: [x])
# # some files are too small/corrupt
# unique_img_ids['file_size_kb'] = unique_img_ids['ImageId'].map(lambda c_img_id: 
#                                                                os.stat(os.path.join(train_image_dir, 
#                                                                                     c_img_id)).st_size/1024)
# unique_img_ids = unique_img_ids[unique_img_ids['file_size_kb']>50] # keep only 50kb files
# unique_img_ids['file_size_kb'].hist()
# masks.drop(['ships'], axis=1, inplace=True)
# unique_img_ids.sample(5)

In [10]:
# from sklearn.model_selection import train_test_split
# train_ids, valid_ids = train_test_split(unique_img_ids, 
#                  test_size = 0.3, 
#                  stratify = unique_img_ids['ships'])
# train_df = pd.merge(masks, train_ids)
# valid_df = pd.merge(masks, valid_ids)
# print(train_df.shape[0], 'training masks')
# print(valid_df.shape[0], 'validation masks')

In [12]:
csv_file = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [15]:
def split_datasets(csv_file):
    train, test = train_test_split(csv_file, test_size=0.01)
    train, val = train_test_split(train, test_size=0.01)
    return train, val, test

### Examine Number of Ship Images
Here we examine how often ships appear and replace the ones without any ships with 0

In [16]:
# train_df['ships'].hist()

# Undersample Empty Images
Here we undersample the empty images to get a better balanced group with more ships to try and segment

In [17]:
# train_df['grouped_ship_count'] = train_df['ships'].map(lambda x: (x+1)//2).clip(0, 7)
# def sample_ships(in_df, base_rep_val=1500):
#     if in_df['ships'].values[0]==0:
#         return in_df.sample(base_rep_val//3) # even more strongly undersample no ships
#     else:
#         return in_df.sample(base_rep_val, replace=(in_df.shape[0]<base_rep_val))
    
# balanced_train_df = train_df.groupby('grouped_ship_count').apply(sample_ships)
# balanced_train_df['ships'].hist(bins=np.arange(10))

# Decode all the RLEs into Images
We make a generator to produce batches of images

In [142]:
def make_image_gen(in_df, batch_size = BATCH_SIZE):
    input_imgs = in_df['fixed_inputs'].values.tolist()
    mask_imgs = in_df['mask_paths'].values.tolist()
    all_batches = [(input_imgs[x], mask_imgs[x]) for x in range(len(input_imgs))]
    out_rgb = []
    out_mask = []
    while True:
        np.random.shuffle(all_batches)
        count = 0
        for c_img_id, c_masks in all_batches:
            rgb_path = c_img_id
            c_img = imread(rgb_path)
            c_mask = imread(c_masks).reshape(768, 768, 1)
            if IMG_SCALING is not None:
                c_img = c_img[::IMG_SCALING[0], ::IMG_SCALING[1]]
                c_mask = c_mask[::IMG_SCALING[0], ::IMG_SCALING[1]]
            out_rgb += [c_img]
            out_mask += [c_mask]
            if len(out_rgb)>=batch_size:
                yield np.stack(out_rgb, 0)/255.0, np.stack(out_mask, 0)
                out_rgb, out_mask=[], []

In [143]:
train_gen = make_image_gen(train)
train_x, train_y = next(train_gen)
print('x', train_x.shape, train_x.min(), train_x.max())
print('y', train_y.shape, train_y.min(), train_y.max())

x (4, 768, 768, 3) 0.0 0.8196078431372549
y (4, 768, 768, 1) 0 0


In [121]:
# fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (30, 10))
# batch_rgb = montage_rgb(train_x)
# batch_seg = montage(train_y[:, :, :, 0])
# ax1.imshow(batch_rgb)
# ax1.set_title('Images')
# ax2.imshow(batch_seg)
# ax2.set_title('Segmentations')
# ax3.imshow(mark_boundaries(batch_rgb, 
#                            batch_seg.astype(int)))
# ax3.set_title('Outlined Ships')
# fig.savefig('overview.png')

# Make the Validation Set

In [124]:
valid_x, valid_y = next(make_image_gen(val, 4))
print(valid_x.shape, valid_y.shape)

(4, 768, 768, 3) (4, 768, 768)


# Augment Data

In [125]:
from keras.preprocessing.image import ImageDataGenerator

2023-03-30 18:35:40.141905: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 18:35:40.302888: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-30 18:35:40.823716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-30 18:35:40.823788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [145]:
from keras.preprocessing.image import ImageDataGenerator
# dg_args = dict(featurewise_center = False, 
#                   samplewise_center = False,
#                   rotation_range = 15, 
#                   width_shift_range = 0.1, 
#                   height_shift_range = 0.1, 
#                   shear_range = 0.01,
#                   zoom_range = [0.9, 1.25],  
#                   horizontal_flip = True, 
#                   vertical_flip = True,
#                   fill_mode = 'reflect',
#                    data_format = 'channels_last')
# # brightness can be problematic since it seems to change the labels differently from the images 
# if AUGMENT_BRIGHTNESS:
#     dg_args[' brightness_range'] = [0.5, 1.5]
# image_gen = ImageDataGenerator(**dg_args)

# if AUGMENT_BRIGHTNESS:
#     dg_args.pop('brightness_range')
# label_gen = ImageDataGenerator(**dg_args)

image_gen = ImageDataGenerator()
label_gen = ImageDataGenerator()

def create_aug_gen(in_gen, seed = None):
    # np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    for in_x, in_y in in_gen:
        # seed = np.random.choice(range(9999))
        # keep the seeds syncronized otherwise the augmentation to the images is different from the masks
        g_x = image_gen.flow(255*in_x, 
                             batch_size = in_x.shape[0], 
                             seed = seed, 
                             shuffle=True)
        g_y = label_gen.flow(in_y, 
                             batch_size = in_x.shape[0], 
                             seed = seed, 
                             shuffle=True)

        yield next(g_x)/255.0, next(g_y)

In [157]:
cur_gen = create_aug_gen(train_gen)
t_x, t_y = next(cur_gen)
print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())
# only keep first 9 samples to examine in detail
t_x = t_x[:9]
t_y = t_y[:9]
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
# ax1.imshow(montage_rgb(t_x), cmap='gray')
# ax1.set_title('images')
# ax2.imshow(montage(t_y[:, :, :, 0]), cmap='gray_r')
# ax2.set_title('ships')

x (4, 768, 768, 3) float32 0.0 1.0
y (4, 768, 768, 1) float32 0.0 255.0


In [158]:
gc.collect()

2455784

# Build a Model
Here we use a slight deviation on the U-Net standard

In [172]:
NET_SCALING

In [173]:
UPSAMPLE_MODE

'SIMPLE'

In [204]:
import tensorflow as tf
from tensorflow import keras

In [199]:
train['fixed_inputs'].values.tolist()[2]

'../../ml-data-training/ship_segmentation_data/train_v2/bafd2ba76.jpg'

In [200]:
img = np.array(PIL.Image.open('../../ml-data-training/ship_segmentation_data/train_v2/bafd2ba76.jpg')).reshape(1, 768, 768, 3)

In [218]:
y = keras.layers.UpSampling2D(size=(2, 2))(img)

In [219]:
y

<tf.Tensor: shape=(1, 1536, 1536, 3), dtype=uint8, numpy=
array([[[[ 1,  3, 26],
         [ 1,  3, 26],
         [ 2,  4, 27],
         ...,
         [13, 11, 35],
         [ 8,  6, 28],
         [ 8,  6, 28]],

        [[ 1,  3, 26],
         [ 1,  3, 26],
         [ 2,  4, 27],
         ...,
         [13, 11, 35],
         [ 8,  6, 28],
         [ 8,  6, 28]],

        [[ 0,  1, 24],
         [ 0,  1, 24],
         [ 0,  2, 25],
         ...,
         [ 7,  7, 31],
         [ 5,  3, 25],
         [ 5,  3, 25]],

        ...,

        [[ 2,  6, 33],
         [ 2,  6, 33],
         [ 0,  4, 31],
         ...,
         [ 2,  3, 34],
         [ 2,  3, 34],
         [ 2,  3, 34]],

        [[ 1,  5, 32],
         [ 1,  5, 32],
         [ 0,  4, 31],
         ...,
         [ 4,  5, 36],
         [ 3,  4, 35],
         [ 3,  4, 35]],

        [[ 1,  5, 32],
         [ 1,  5, 32],
         [ 0,  4, 31],
         ...,
         [ 4,  5, 36],
         [ 3,  4, 35],
         [ 3,  4, 35]]]], dty

In [171]:
from keras import models, layers
# Build U-Net model
def upsample_conv(filters, kernel_size, strides, padding):
    return layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)
def upsample_simple(filters, kernel_size, strides, padding):
    return layers.UpSampling2D(strides)

if UPSAMPLE_MODE=='DECONV':
    upsample=upsample_conv
else:
    upsample=upsample_simple
    
# Please change below line shape accordingly 32 x 32 is not working and assuming anything lesser than that will not
# work as well.
input_img = layers.Input((64, 64, 3), name = 'RGB_Input')
pp_in_layer = input_img
if NET_SCALING is not None:
    pp_in_layer = layers.AvgPool2D(NET_SCALING)(pp_in_layer)
    
pp_in_layer = layers.GaussianNoise(GAUSSIAN_NOISE)(pp_in_layer)
pp_in_layer = layers.BatchNormalization()(pp_in_layer)

c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (pp_in_layer)
c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = layers.MaxPooling2D((2, 2)) (c1)

c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = layers.MaxPooling2D((2, 2)) (c2)

c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = layers.MaxPooling2D((2, 2)) (c3)

c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = layers.MaxPooling2D(pool_size=(2, 2)) (c4)


c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = upsample(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = layers.concatenate([u6, c4])
c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = upsample(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = layers.concatenate([u7, c3])
c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = upsample(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = layers.concatenate([u8, c2])
c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = upsample(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = layers.concatenate([u9, c1], axis=3)
c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

d = layers.Conv2D(1, (1, 1), activation='sigmoid') (c9)
d = layers.Cropping2D((EDGE_CROP, EDGE_CROP))(d)
d = layers.ZeroPadding2D((EDGE_CROP, EDGE_CROP))(d)
if NET_SCALING is not None:
    d = layers.UpSampling2D(NET_SCALING)(d)

seg_model = models.Model(inputs=[input_img], outputs=[d])
seg_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 RGB_Input (InputLayer)         [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 gaussian_noise_4 (GaussianNois  (None, 64, 64, 3)   0           ['RGB_Input[0][0]']              
 e)                                                                                               
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 64, 64, 3)   12          ['gaussian_noise_4[0][0]']       
 rmalization)                                                                                     
                                                                                            

In [163]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
def dice_p_bce(in_gt, in_pred):
    return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)
def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)
seg_model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_p_bce, metrics=[dice_coef, 'binary_accuracy', true_positive_rate])

In [164]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('seg_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_dice_coef', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, 
                                   patience=3, 
                                   verbose=1, mode='max', epsilon=0.0001, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="val_dice_coef", 
                      mode="max", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]


In [167]:
train.shape

(188723, 4)

In [168]:
step_count = min(MAX_TRAIN_STEPS, train.shape[0]//BATCH_SIZE)
aug_gen = create_aug_gen(make_image_gen(train))
loss_history = [seg_model.fit_generator(aug_gen, 
                             steps_per_epoch=step_count, 
                             epochs=NB_EPOCHS, 
                             validation_data=(valid_x, valid_y),
                             callbacks=callbacks_list,
                            workers=1 # the generator is not very thread safe
                                       )]

/tmp/ipykernel_355982/3169141231.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  loss_history = [seg_model.fit_generator(aug_gen,


Epoch 1/5
 21/200 [==>...........................] - ETA: 10:51 - loss: -0.0921 - dice_coef: 0.0929 - binary_accuracy: 0.6340 - true_positive_rate: nan

KeyboardInterrupt: 

In [ ]:
def show_loss(loss_history):
    epich = np.cumsum(np.concatenate(
        [np.linspace(0.5, 1, len(mh.epoch)) for mh in loss_history]))
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(22, 10))
    _ = ax1.plot(epich,
                 np.concatenate([mh.history['loss'] for mh in loss_history]),
                 'b-',
                 epich, np.concatenate(
            [mh.history['val_loss'] for mh in loss_history]), 'r-')
    ax1.legend(['Training', 'Validation'])
    ax1.set_title('Loss')

    _ = ax2.plot(epich, np.concatenate(
        [mh.history['true_positive_rate'] for mh in loss_history]), 'b-',
                     epich, np.concatenate(
            [mh.history['val_true_positive_rate'] for mh in loss_history]),
                     'r-')
    ax2.legend(['Training', 'Validation'])
    ax2.set_title('True Positive Rate\n(Positive Accuracy)')
    
    _ = ax3.plot(epich, np.concatenate(
        [mh.history['binary_accuracy'] for mh in loss_history]), 'b-',
                     epich, np.concatenate(
            [mh.history['val_binary_accuracy'] for mh in loss_history]),
                     'r-')
    ax3.legend(['Training', 'Validation'])
    ax3.set_title('Binary Accuracy (%)')
    
    _ = ax4.plot(epich, np.concatenate(
        [mh.history['dice_coef'] for mh in loss_history]), 'b-',
                     epich, np.concatenate(
            [mh.history['val_dice_coef'] for mh in loss_history]),
                     'r-')
    ax4.legend(['Training', 'Validation'])
    ax4.set_title('DICE')

show_loss(loss_history)

In [ ]:
seg_model.load_weights(weight_path)
seg_model.save('seg_model.h5')

In [ ]:
pred_y = seg_model.predict(valid_x)
print(pred_y.shape, pred_y.min(), pred_y.max(), pred_y.mean())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (10, 10))
ax.hist(pred_y.ravel(), np.linspace(0, 1, 10))
ax.set_xlim(0, 1)
ax.set_yscale('log', nonposy='clip')

# Prepare Full Resolution Model
Here we account for the scaling so everything can happen in the model itself

In [ ]:
if IMG_SCALING is not None:
    fullres_model = models.Sequential()
    fullres_model.add(layers.AvgPool2D(IMG_SCALING, input_shape = (None, None, 3)))
    fullres_model.add(seg_model)
    fullres_model.add(layers.UpSampling2D(IMG_SCALING))
else:
    fullres_model = seg_model
fullres_model.save('fullres_model.h5')

# Run the test data

In [ ]:
test_paths = os.listdir(test_image_dir)
print(len(test_paths), 'test images found')

In [ ]:
fig, m_axs = plt.subplots(20, 2, figsize = (10, 40))
[c_ax.axis('off') for c_ax in m_axs.flatten()]
for (ax1, ax2), c_img_name in zip(m_axs, test_paths):
    c_path = os.path.join(test_image_dir, c_img_name)
    c_img = imread(c_path)
    first_img = np.expand_dims(c_img, 0)/255.0
    first_seg = fullres_model.predict(first_img)
    ax1.imshow(first_img[0])
    ax1.set_title('Image')
    ax2.imshow(first_seg[0, :, :, 0], vmin = 0, vmax = 1)
    ax2.set_title('Prediction')
fig.savefig('test_predictions.png')

# Submission
Since gneerating the submission takes a long time and quite a bit of memory we run it in a seperate kernel located at https://www.kaggle.com/kmader/from-trained-u-net-to-submission-part-2 
That kernel takes the model saved in this kernel and applies it to all the test data